# Collecting Data

In [ ]:
!git clone https://github.com/alexeygrigorev/clothing-dataset-small

# Import images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img

%matplotlib inline

In [ ]:
img = load_img('./clothing-dataset-small/train/t-shirt/5f0a3fa0-6a3d-4b68-b213-72766a643de7.jpg', 
            target_size=(299, 299))
img

# Image Data Generator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_gen = {}
for s in ['train', 'validation', 'test']:
  data_gen[s] = ImageDataGenerator(preprocessing_function=preprocess_input, ).flow_from_directory(
      f'./clothing-dataset-small/{s}',
      target_size=(150, 150),
      batch_size=32

## Preview images

In [ ]:
import os
from skimage import io

path_tr = "/content/clothing-dataset-small/train"
path_val = "/content/clothing-dataset-small/validation"

for path, title in zip([path_tr, path_val], ['Training images', 'Validation Images']):
  img_folders = os.listdir(path)
  print(f'{"---"*10}\n{title}')
  for i in img_folders:
      images_path = os.path.join(path , i)
      fig , axs = plt.subplots(1, 5 , figsize = (10 ,8))
      fig.suptitle(i, y=0.62, fontsize=12)
      for ax in axs:
          fname = os.listdir(images_path)
          ran = fname[np.random.randint(0 ,len(fname)-1)]
          pic = io.imread(os.path.join(images_path , ran))
          ax.set_xticks([])
          ax.set_yticks([])
          ax.imshow(pic)
  plt.show();

In [ ]:
data_gen['train'].class_indices

In [ ]:
X, y = next(data_gen['train'])

# Transfer Learning

In [ ]:
from tensorflow.keras.applications.xception import Xception, preprocess_input, decode_predictions
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
base_model = Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(150, 150, 3)
)

base_model.trainable = False



In [ ]:
inputs = keras.Input(shape=(150, 150, 3))

base = base_model(inputs, training=False)
pooling = keras.layers.GlobalAveragePooling2D()(base)

outputs = keras.layers.Dense(10, activation='linear')(pooling)


model = keras.Model(inputs, outputs)


In [ ]:
model.summary()

## Training

In [ ]:
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
loss = keras.losses.CategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])


In [ ]:
history = model.fit(data_gen['train'], epochs=10,
                    validation_data=data_gen['validation'])

In [ ]:
history.history.keys()

In [ ]:
for metric in ['accuracy', 'loss']:
    plt.plot(np.arange(1, epochs+1), history.history[metric], label='train')
    plt.plot(np.arange(1, epochs+1), history.history[f'val_{metric}'], label='val')
    plt.xticks(np.arange(1, epochs+1))
    plt.legend()
    plt.title(metric)
    plt.show();

## Evaluation

In [ ]:
y_pred_tr = model.predict(data_gen['train'])
y_pred_val = model.predict(data_gen['validation'])
y_pred_te = model.predict(data_gen['test'])

In [ ]:
def evaluate(ds, y_pred):
    y_val = ds.classes
    labels = np.array(list(ds.class_indices.keys()))

    cm = confusion_matrix(y_val, y_pred.argmax(axis=1))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot() # cmap=plt.cm.Blues
    plt.show();
    
    print(classification_report(y_val, y_pred.argmax(axis=1), target_names=labels))

In [ ]:
for data, y_pred in zip(['train', 'validation', 'test'], [y_pred_tr, y_pred_val, y_pred_te]):
    print(f"---------------{data}---------------")
    evaluate(data_gen[data], y_pred)
    print()

In [ ]:
train_results = pd.DataFrame({'path':pd.Series(data_gen['train'].filepaths), 
              'actual':data_gen['train'].classes, 
              'predicted':y_pred_tr.argmax(axis=1)})
train_results.head()

In [ ]:
labels = np.array(list(data_gen['train'].class_indices.keys()))

for cls in train_results.actual.unique():
    samples = train_results[train_results.actual == cls].sample(5)
    fig , axs = plt.subplots(1, 5 , figsize = (10 ,8))
    fig.suptitle(labels[cls], y=0.62, fontsize=12)
    for i, ax in enumerate(axs):
        pic = io.imread(samples.iloc[i,0])
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlabel(f"Prediction\n{labels[samples.iloc[i,2]]}")
        ax.imshow(pic);

In [ ]:
model.evaluate(data_gen['test'])
# [loss, acc]

# Prediction

## Predict 1 image

In [ ]:
img = load_img('./clothing-dataset-small/train/t-shirt/5f0a3fa0-6a3d-4b68-b213-72766a643de7.jpg', 
            target_size=(150, 150))
img

In [ ]:
img_arr = np.array(img)
img_1 = np.array([img_arr])
preprocessed_img_1 = preprocess_input(img_1)
img_pred = model.predict(preprocessed_img_1)
img_pred

In [ ]:
classes = ['dress',
            'hat',
            'longsleeve',
            'outwear',
            'pants',
            'shirt',
            'shoes',
            'shorts',
            'skirt',
            't-shirt']

dict(zip(classes, img_pred[0]))

# Save Model

## Save Keras model

In [ ]:
model.save_weights('clothes-model.h5', save_format='h5')

## Convert Keras to TF-Lite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('clothing-model.tflite', 'wb') as f_out:
  f_out.write(tflite_model)

## Load tflite

In [ ]:
import tensorflow.lite as tflite
# load the tflite model
interpreter = tflite.Interpreter(model_path='clothing-model.tflite')
# load the weights from the model to memory
interpreter.allocate_tensors()

In [ ]:
interpreter.get_input_details()

In [ ]:
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
# insitialize the input
interpreter.set_tensor(input_index, preprocessed_img_1)
# invoke the model
interpreter.invoke()
# collect output
pred = interpreter.get_tensor(output_index)
pred

In [ ]:
classes = ['dress',
            'hat',
            'longsleeve',
            'outwear',
            'pants',
            'shirt',
            'shoes',
            'shorts',
            'skirt',
            't-shirt']

dict(zip(classes, pred[0]))

## Remove TF dependency

In [ ]:
# preprocessed_img_1 = preprocess_input(img_1)
def preprocess_input_img(x):
    x /= 127.5
    x -= 1.
    return x

In [ ]:
from PIL import Image
with Image.open('./clothing-dataset-small/train/t-shirt/5f0a3fa0-6a3d-4b68-b213-72766a643de7.jpg') as img:
  img = img.resize((150, 150), Image.NEAREST)

img_arr = np.array(img, dtype='float32')
img_1 = np.array([img_arr]) 
preprocessed_img_1 = preprocess_input_img(img_1)

In [ ]:
import tensorflow.lite as tflite
# load the tflite model
interpreter = tflite.Interpreter(model_path='clothing-model.tflite')
# load the weights from the model to memory
interpreter.allocate_tensors()

In [ ]:
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
# insitialize the input
interpreter.set_tensor(input_index, preprocessed_img_1)
# invoke the model
interpreter.invoke()
# collect output
pred = interpreter.get_tensor(output_index)
pred

In [ ]:
classes = ['dress',
            'hat',
            'longsleeve',
            'outwear',
            'pants',
            'shirt',
            'shoes',
            'shorts',
            'skirt',
            't-shirt']

dict(zip(classes, pred[0]))

# Parameter Tuning